# 0) The HALE Aircraft Model Class

The following Jupyter notebook will explain briefly how the HALE aircraft model class is set up and how to get it ready for a wide variety of analyses which will be explained in detail in subsequent chapters. 

This example will show how to manipulate the class to get the appropriate `.fem.h5` and `.aero.h5` file inputs that SHARPy needs.

The HALE aircraft model is hosted in an independent repository at [http://github.com/ngoiz/model-hale](http://github.com/ngoiz/model-hale). The repo can be cloned and the variables loaded by running `source <path_to model-hale>/bin/hale_vars.sh` (just like in SHARPy).

For this tutorial, it is required that you have the HALE aircraft model and SHARPy.

In [1]:
import aircraft  # This is the HALE aircraft. If ModuleNotFound, make sure you have correctly loaded the model-hale variables

In [2]:
import numpy as np
import sharpy.utils.algebra as algebra

The HALE class consists of a top-level class `aircraft.Hale()` which will generate a structure (`structure.HaleStructure()`) and an aerodynamic model `aero.HaleAero()`.

Let's first instantiate the top-level class, which takes a `case_name`, `cases_route` and an `output_route` as constructor arguments.

In [3]:
cases_route = './cases/'
output_route = './output/'

case_name = 'hale'

hale = aircraft.Hale(case_name, cases_route, output_route)

It is good practice to remove any previous files for the same `case_name`. This functionality is built-in with

In [8]:
hale.clean()

We then start by initialising the structural model. All the structural properties are defined in `structure.py` and can easily be changed. The only two parameters that need to be provided are `sigma` which is a stiffenning factor and `n_elem_multiplier` which is a multiplication factor to control how refined the model is. These can be provided through key-word arguments to the method `Hale().init_structure()`

In [9]:
hale.init_structure(sigma=1, n_elem_multiplier=1.5)

Let's proceed now to similarly initiate the aerodynamics. The only parameter we need to provide at this stage is the number of chordwise panels `m`.

In [10]:
hale.init_aero(m=4)

The only remaining information that needs to go in the `.h5` files are the flight control surfaces deflections and thrust. Control surface deflections are given in `rad`.

In [11]:
hale.set_flight_controls(thrust=5, elevator=1. * np.pi / 180, rudder=0.)

These concludes all parameters that are needed for the `.fem.h5` and `.aero.h5` files. Don't worry, a lot more info will be required for the simulations, but those params go in the simulation input file `.sharpy`. We can now proceed and generate these files which will be saved to our chosen `cases_directory`

In [12]:
hale.generate()

The `.h5` files have been generated. 

## Generating the simulation input file

This is done through a dictionary and will not present any difficulty for simple simulations. More advanced analyses which will be shown later to involve more convoluted settings which will be described in detail at that time. For the moment, let's demonstrate how to parse the settings dictionary to the `hale` object and generate our `.sharpy` file to create a simple visualisation in Paraview.

The simulation will consist of the HALE aircraft at a non-zero angle of attack.

In [14]:
alpha = 5 * np.pi / 180

In [21]:
settings = {}

settings['SHARPy'] = {'case': hale.case_name,
                      'route': hale.case_route,
                      'flow': ['BeamLoader', 
                               'AerogridLoader',
                               'BeamPlot',
                               'AerogridPlot'
                              ],
                      'write_screen': 'on',
                      'write_log': 'on',
                      'log_folder': hale.output_route,
                      'log_file': hale.case_name + '.log'}

The settings for these solvers are relatively straight forward and documented extensively in the SHARPy documentation
and examples thus we will tackle them swiftly.


In [22]:
settings['BeamLoader'] = {'unsteady': 'on',
                              'orientation': algebra.euler2quat(np.array([0.,
                                                                          alpha,
                                                                          0.]))}


As opposed to the earlier versions of SHARPy though, the wake is now initialised in `AerogridLoader`. Thus, despite this case just being inteded for visualisation, we are going to define the wake for which we need the free stream velocity and time step increment. For more details on the wake shape generation, please refer to [https://ic-sharpy.readthedocs.io/en/master/includes/generators/straightwake/StraightWake.html](https://ic-sharpy.readthedocs.io/en/master/includes/generators/straightwake/StraightWake.html). We will refer to this later on when we discuss cases where the CFL is not equal to one and the wake panels change in size to reduce the number of wake panels.

In [26]:
u_inf = 10
dt = hale.aero.chord_main / hale.aero.m / u_inf  # This achieves a CFL = 1 condition

settings['AerogridLoader'] = {'unsteady': 'on',
                                  'aligned_grid': 'on',
                                  'mstar': 10,
                                  'wake_shape_generator': 'StraightWake',
                                  'wake_shape_generator_input': {
                                      'u_inf': u_inf,
                                      'u_inf_direction': [1., 0., 0.],
                                      'dt': dt,
                                  },
                                  }

settings['BeamPlot'] = {}

settings['AerogridPlot'] = {'include_rbm': 'off',
                            'include_applied_forces': 'on',
                            'minus_m_star': 0,
                            'u_inf': u_inf
                            }

The settings can now be parsed to the `hale` object to generate the `.sharpy` file

In [27]:
hale.create_settings(settings)

If we wish to run the case from within a script rather than the command line, a `run()` method is provided.

In [28]:
hale.run()

--------------------------------------------------------------------------------
            ######  ##     ##    ###    ########  ########  ##    ##
           ##    ## ##     ##   ## ##   ##     ## ##     ##  ##  ##
           ##       ##     ##  ##   ##  ##     ## ##     ##   ####
            ######  ######### ##     ## ########  ########     ##
                 ## ##     ## ######### ##   ##   ##           ##
           ##    ## ##     ## ##     ## ##    ##  ##           ##
            ######  ##     ## ##     ## ##     ## ##           ##
--------------------------------------------------------------------------------
Aeroelastics Lab, Aeronautics Department.
    Copyright (c), Imperial College London.
    All rights reserved.
    License available at https://github.com/imperialcollegelondon/sharpy
Running SHARPy from /home/ng213/2TB/KK_AirbusHALE/00_HALE_model
SHARPy being run is in /home/ng213/2TB/KK_AirbusHALE/src/sharpy
The branch being run is dev_remove_istates
The version and

The resulting output files are located in the output folder